# 加载 Qwen-1_8B-Chat-Int4 模型

In [1]:
from modelscope import snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer

# 没下载模型就执行下面这句，因为huggingface没办法直接连，科学上网也没用，所以换成魔塔的了
model_dir = snapshot_download('qwen/Qwen-1_8B-Chat-Int4')
# 下载好模型的可以执行下面这句，直接拿到本地的模型目录
model_dir = "/home/hanyecyh/.cache/modelscope/hub/models/qwen/Qwen-1_8B-Chat-Int4"

# 加载分词器和模型
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",
    trust_remote_code=True
).eval()

/home/hanyecyh/miniconda3/envs/QwenSFT/lib/python3.8/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/home/hanyecyh/miniconda3/envs/QwenSFT/lib/python3.8/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


2025-03-05 11:46:25,201 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
/home/hanyecyh/miniconda3/envs/QwenSFT/lib/python3.8/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:411: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, qweight, scales, qzeros, g_idx, bits, maxq):
/home/hanyecyh/miniconda3/envs/QwenSFT/lib/python3.8/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:419: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/home/hanyecyh/miniconda3/envs/QwenSFT/lib/python3.8/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=t

# 提示词

In [5]:
Q='早上起来一直有过敏性鼻炎发作，怎么办？'

prompt_template='''
给定一句话：“%s”，请你按步骤要求工作。

步骤1：识别用户问题中遇到身体上的常见症状，如感冒、发烧等。
步骤2：根据问题，生成JSON格式字符串，格式为{"①":应对症状的一种做法, "②":针对症状的另一种做法}，根据具体症状给出一种或多种解决方案，分点作答

请问，这个JSON字符串是：

'''
prompt = prompt_template%(Q,)
# print(prompt)  # 给模型的命令

resp, hist = model.chat(tokenizer, prompt, history=None)
print(resp)

由于没有提供具体的问题和对应的解决方案，我将以一个常见的过敏性鼻炎为例来回答这个问题。假设问题如下：“早上起来一直有过敏性鼻炎发作，怎么办？”在这种情况下，我们可以给出以下的JSON字符串：

```json
{
  "①": "避免接触可能引起过敏反应的物质",
  "②": "服用抗过敏药物"
}
```

此JSON字符串中，“①”表示对过敏症状的具体治疗措施，比如“避免接触可能引起过敏反应的物质”。而“②”则表示使用抗过敏药物作为辅助治疗方式。

如果你能提供更具体的场景或问题，我会很乐意为你进一步提供相应的JSON字符串。


# 批量整理训练所需数据

In [6]:
train_data = []
flag0 = True
flag1 = True
flag2 = False
Q = "question"
A = ""
i = 0

with open('medical.txt','r',encoding='utf-8') as fp:
    for line in fp:
        if 'QUESTION' in line:
            flag1 = True
            flag2 = False
            # 第一次的话前面没有样本，直接跳过
            if i == 0:
                continue
            # 添加一个example(本次QUESTION不是第一次，上面还有一个example)
            example={
                'id': 'identity_{}'.format(i),
                'conversations':[
                    {
                        'from':'user',
                        'value':prompt_template%(Q,),
                    },
                    {
                        'from':'assistant',
                        'value':A,
                    },
                ]
            }
            train_data.append(example)
            # A不能无限累加，要重新置0
            A = ""
        elif 'ANSWER' in line:
            flag1 = False
            flag2 = True
        elif flag1:
            Q = line
        else:
            A += line

        i+=1

    # 最后退出时，由于后面没QUESTION可以后处理，因此在此处还需要处理最后一批数据
    example={
        'id': 'identity_{}'.format(i),
        'conversations':[
            {
                'from':'user',
                'value':prompt_template%(Q,),
            },
            {
                'from':'assistant',
                'value':A,
            },
        ]
    }
    train_data.append(example)

# 导出训练数据

In [7]:
import json
with open('medical_train.txt', 'w', encoding='utf-8') as fp:
    fp.write(json.dumps(train_data))

# 微调模型，生成到output_qwen
bash 模型脚本   模型路径   训练文件  
bash finetune/finetune_qlora_single_gpu.sh -m /home/hanyecyh/.cache/modelscope/hub/models/qwen/Qwen-1_8B-Chat-Int4 -d /home/hanyecyh/python_code/Qwen/medical_train.txt

# 加载SFT后的模型

In [8]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    'output_qwen', # path to the output directory
    device_map="auto",
    trust_remote_code=True
).eval()

Try importing flash-attention for faster inference...
Some weights of the model checkpoint at /home/hanyecyh/python_code/Qwen/model1_8 were not used when initializing QWenLMHeadModel: ['transformer.h.11.mlp.w2.bias', 'transformer.h.7.mlp.c_proj.bias', 'transformer.h.9.mlp.c_proj.bias', 'transformer.h.22.mlp.w2.bias', 'transformer.h.3.mlp.c_proj.bias', 'transformer.h.0.attn.c_proj.bias', 'transformer.h.17.attn.c_proj.bias', 'transformer.h.19.mlp.w1.bias', 'transformer.h.16.mlp.w2.bias', 'transformer.h.5.attn.c_proj.bias', 'transformer.h.7.mlp.w2.bias', 'transformer.h.11.attn.c_proj.bias', 'transformer.h.20.mlp.c_proj.bias', 'transformer.h.5.mlp.w1.bias', 'transformer.h.12.attn.c_proj.bias', 'transformer.h.15.mlp.w1.bias', 'transformer.h.10.attn.c_proj.bias', 'transformer.h.7.mlp.w1.bias', 'transformer.h.6.mlp.c_proj.bias', 'transformer.h.0.mlp.c_proj.bias', 'transformer.h.21.mlp.w2.bias', 'transformer.h.17.mlp.w1.bias', 'transformer.h.0.mlp.w1.bias', 'transformer.h.1.attn.c_proj.bias', 

# 利用微调后的模型，重新回答若干问题

In [9]:
model.generation_config.top_p=0 # 只选择概率最高的token

Q_list=['不小心怀孕了要怎么避孕？','我早上起来有过敏性鼻炎怎么办','最近嘴唇和手脚一直干裂','最近口腔感觉不卫生']
for Q in Q_list:
    prompt=prompt_template%(Q,)
    print(prompt)
    A,hist=model.chat(tokenizer,prompt,history=None)
    print('Q:%s\nA:%s\n'%(Q,A))


给定一句话：“不小心怀孕了要怎么避孕？”，请你按步骤要求工作。

步骤1：识别用户问题中遇到身体上的常见症状，如感冒、发烧等。
步骤2：根据问题，生成JSON格式字符串，格式为{"①":应对症状的一种做法, "②":针对症状的另一种做法}，根据具体症状给出一种或多种解决方案，分点作答

请问，这个JSON字符串是：


Q:不小心怀孕了要怎么避孕？
A:症状:感冒
应对症状:休息并多喝水
应对症状:服用解热药
应对症状:就医检查
症状:发烧
应对症状:多喝水
应对症状:服用退烧药
症状:恶心
应对症状:休息并多喝水
应对症状:就医检查
症状:呕吐
应对症状:休息并多喝水
应对症状:就医检查


给定一句话：“我早上起来有过敏性鼻炎怎么办”，请你按步骤要求工作。

步骤1：识别用户问题中遇到身体上的常见症状，如感冒、发烧等。
步骤2：根据问题，生成JSON格式字符串，格式为{"①":应对症状的一种做法, "②":针对症状的另一种做法}，根据具体症状给出一种或多种解决方案，分点作答

请问，这个JSON字符串是：


Q:我早上起来有过敏性鼻炎怎么办
A:症状:过敏性鼻炎
应对症状:避免接触过敏源
针对症状:服用抗过敏药
解决方案:
1. 保持室内空气清新，避免尘螨和花粉等过敏原。
2. 饮食上应避免食用可能引发过敏的食物，如海鲜、牛奶等。
3. 使用空气净化器可以有效减少室内过敏原。
4. 如果症状严重，可考虑就医治疗。


给定一句话：“最近嘴唇和手脚一直干裂”，请你按步骤要求工作。

步骤1：识别用户问题中遇到身体上的常见症状，如感冒、发烧等。
步骤2：根据问题，生成JSON格式字符串，格式为{"①":应对症状的一种做法, "②":针对症状的另一种做法}，根据具体症状给出一种或多种解决方案，分点作答

请问，这个JSON字符串是：


Q:最近嘴唇和手脚一直干裂
A:症状:嘴唇和手脚干裂
应对症状:保持充足的水分摄入
针对症状:使用润唇膏或者保湿霜
解决方案:
1.多喝水，保持身体水分充足
2.避免口渴时过度饮水
3.涂抹润唇膏或者保湿霜，保持嘴唇和手脚湿润


给定一句话：“最近口腔感觉不卫生”，请你按步骤要求工作。

步骤1：识别用户问题中遇到身体上的常见症状，如感冒、发烧等。
步骤2：根据问题，生成JSON格式字符串，格式为{"①":应对症状的一种做法, 

# 单问题(无提示词)

In [10]:
prompt='最近嘴唇和手脚一直干裂？'
resp,hist=model.chat(tokenizer,prompt,history=None)
print('Q:%s\nA:%s\n'%(prompt,resp))

Q:最近嘴唇和手脚一直干裂？
A:嘴唇和手脚干燥裂口可能是由于多种原因引起的，包括缺水、缺乏营养、皮肤疾病等。以下是一些可能的解决方案：

1. 补水：每天喝足够的水可以帮助保持身体水分平衡，缓解干燥症状。

2. 保湿霜：选择含有天然油脂或滋润成分的保湿霜，如乳液、润唇膏等，涂抹在嘴唇和手脚上，可以提供额外的保护和滋润。

3. 饮食调整：多吃富含维生素A、C和E的食物，如胡萝卜、柑橘类水果、坚果等，这些食物有助于维护皮肤健康。

4. 其他护理产品：使用含有甘油、尿囊素等保湿成分的产品，如护手霜、润肤露等，也可以帮助改善干燥状况。

5. 如果以上方法都不能解决问题，建议就医检查，以排除其他可能的病因。

请注意，每个人的体质和环境都不同，所以可能需要尝试不同的方法才能找到最适合自己的解决方案。



# 单问题(有提示词)

In [11]:
Q='最近嘴唇和手脚一直干裂？'
prompt=prompt_template%(Q,)
resp,hist=model.chat(tokenizer,prompt,history=None)
print('Q:%s\nA:%s\n'%(prompt,resp))

Q:
给定一句话：“最近嘴唇和手脚一直干裂？”，请你按步骤要求工作。

步骤1：识别用户问题中遇到身体上的常见症状，如感冒、发烧等。
步骤2：根据问题，生成JSON格式字符串，格式为{"①":应对症状的一种做法, "②":针对症状的另一种做法}，根据具体症状给出一种或多种解决方案，分点作答

请问，这个JSON字符串是：


A:症状:嘴唇和手脚干裂
应对症状:保持充足的水分摄入
针对症状:使用润唇膏或者保湿霜
解决方案:
1.多喝水，保持身体水分充足
2.避免过度口渴，可以适当喝一些含糖饮料
3.涂抹润唇膏或者保湿霜，保持嘴唇湿润
4.如果症状持续不减或者加重，建议就医

